In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow.keras import regularizers
import random
from tensorflow import keras
from tensorflow.keras.layers import Dropout

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

Using TensorFlow backend.


In [0]:
!pip install xlrd
import io
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload

from google.colab import drive 

auth.authenticate_user()
drive_service = build('drive', 'v3')

In [0]:
### This file retrieval code was adapted from starter code provided by the BioE 101 staff.

set_1_train = '12llyDouiydDqzjLo7B6cKHddm5IeoV8f'    # Put helper_function.py id inbetween the quotes
set_1_test     = '1XqBhnJlJQPZfYTrSYJI5FP7HT_7-yu30'    # Put p9.xlsx id inbetween the quotes
set_1_dev    = '1wHLY52QrrQtDNN3fqNQcWRI855wQg96O'    # Put p10.xlsx id inbetween the quotes
set_2_train = '1MixvUK_5XtlbqZlH1nPbm-bTFS99sLwp'    # Put helper_function.py id inbetween the quotes
set_2_test     = '18VEZVO5pi4V8IGTWVrIM7O1yOmvkmZfc'    # Put p9.xlsx id inbetween the quotes
set_2_dev    = '1pYgUWFhEHsOcAsz3WdYNQ4NstSmxA_Cn'    # Put p10.xlsx id inbetween the quotes


file_ids = [set_1_train, set_1_test, set_1_dev, set_2_train, set_2_test, set_2_dev]
file_names = ["set_1_train.tsv", "set_1_test.tsv", "set_1_dev.tsv", "set_2_train.tsv", "set_2_test.tsv", "set_2_dev.tsv"]

for i in list(range(len(file_ids))):
  request = drive_service.files().get_media(fileId=file_ids[i])
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
    _, done = downloader.next_chunk()

  downloaded.seek(0)
  with open(file_names[i], 'wb') as f:
    f.write(downloaded.read())

In [0]:
class_names = ["attB", "attP", "control"]

In [0]:
CENTER_PADDING = False

The code in the following cell for the following functions was adapted from our RNN implementation: load_data(...), pad_zeros(...), get_y(...).

In [0]:
# SET 2

char2index = {'A':[1, 0, 0, 0], 'C':[0, 1, 0, 0], 'G':[0, 0, 1, 0], 'T':[0, 0, 0, 1]}

def load_data(path):
    df = pd.read_csv(path, sep='\t')

    # No duplicates in set 2, but just in case
    attb = list(set([seq for seq in df.loc[df['type'] == "attb"].att]))
    attp = list(set([seq for seq in df.loc[df['type'] == "attp"].att]))
    control = list(set([seq for seq in df.loc[df['type'] == "control"].att]))
    shuf = list(set([seq for seq in df.loc[df['type'] == "shuf"].att]))

    print(path)

    X = [[char2index[char] for char in seq] for seq in attb]
    Y = [[char2index[char] for char in seq] for seq in attp]
    
    Z = [[char2index[char] for char in seq] for seq in control]
    Z += [[char2index[char] for char in seq] for seq in shuf]
    return X, Y, Z

# train_attb, train_attp, train_control = load_data('set2/attx_protein_binding/train.tsv')
# dev_attb, dev_attp, dev_control = load_data('set2/attx_protein_binding/dev.tsv')
# test_attb, test_attp, test_control = load_data('set2/attx_protein_binding/test.tsv')
train_attb, train_attp, train_control = load_data('set_2_train.tsv')
dev_attb, dev_attp, dev_control = load_data('set_2_dev.tsv')
test_attb, test_attp, test_control = load_data('set_2_test.tsv')


def pad_zeros(data, max_pad=160, centered_pad=True):
    padded_data = []
    for row in data:
        if max_pad - len(row) > 0:
            if centered_pad:
                diff = max_pad - len(row)
                left_val = int(diff / 2)
                right_val = int(diff / 2) + int(diff % 2)
                row = [[0, 0, 0, 0]]*left_val + row + [[0, 0, 0, 0]]*right_val
            else:
                row = row + [[0, 0, 0, 0]]*(max_pad - len(row))
        padded_data.append(row)
    return np.array(padded_data)


def get_y(attb, attp, control):
    y_to_return = np.array(len(attb) * [0] + len(attp) * [1] + len(control) * [2])
    return y_to_return.reshape(y_to_return.shape[0], 1)

print(len(train_attb))
print(len(train_attp))
print(len(train_attb + train_attp))
train_x = train_attb + train_attp + train_control
dev_x = dev_attb + dev_attp + dev_control
test_x = test_attb + test_attp + test_control

train_x, train_y = pad_zeros(train_x, centered_pad=CENTER_PADDING), get_y(train_attb, train_attp, train_control)
dev_x, dev_y = pad_zeros(dev_x, centered_pad=CENTER_PADDING), get_y(dev_attb, dev_attp, dev_control)
test_x, test_y = pad_zeros(test_x, centered_pad=CENTER_PADDING), get_y(test_attb, test_attp, test_control)

print(train_y.shape)
print(train_x.shape)
print(train_x.shape)
print(test_x.shape)

set_2_train.tsv
set_2_dev.tsv
set_2_test.tsv
9613
10554
20167
(60501, 1)
(60501, 160, 4)
(60501, 160, 4)
(19662, 160, 4)


In [0]:
#### SET 1 IS FORMATTED DIFF FROM SET 2
#### JUST USE OLD CODE FROM CS_230_CLASSIFIER.IPYNB (ORIGINAL)

char2index = {'A':[1, 0, 0, 0], 'C':[0, 1, 0, 0], 'G':[0, 0, 1, 0], 'T':[0, 0, 0, 1]}

def load_data(path):
    df = pd.read_csv(path, sep='\t')
    print(path)

    # Remove duplicates
    attb = list(set([seq for seq in df.attb]))
    attp = list(set([seq for seq in df.attp]))

    B = [[char2index[char] for char in seq] for seq in attb]
    P = [[char2index[char] for char in seq] for seq in attp]
    return B, P

train_attb, train_attp = load_data('set_1_train.tsv')
dev_attb, dev_attp = load_data('set_1_dev.tsv')
test_attb, test_attp = load_data('set_1_test.tsv')

def pad_zeros(data, max_pad=160, centered_pad=False):
    padded_data = []
    for row in data:
        if max_pad - len(row) > 0:
            if centered_pad:
                diff = max_pad - len(row)
                left_val = int(diff / 2)
                right_val = int(diff / 2) + int(diff % 2)
                row = [[0, 0, 0, 0]]*left_val + row + [[0, 0, 0, 0]]*right_val
            else:
                row = row + [[0, 0, 0, 0]]*(max_pad - len(row))
        padded_data.append(row)
    return np.array(padded_data)

print(len(train_attb))
print(len(train_attp))
print(len(train_attb + train_attp))
train_x = train_attb + train_attp + train_control
dev_x = dev_attb + dev_attp + dev_control
test_x = test_attb + test_attp + test_control

train_x, train_y = pad_zeros(train_x, centered_pad=CENTER_PADDING), get_y(train_attb, train_attp, train_control)
dev_x, dev_y = pad_zeros(dev_x, centered_pad=CENTER_PADDING), get_y(dev_attb, dev_attp, dev_control)
test_x, test_y = pad_zeros(test_x, centered_pad=CENTER_PADDING), get_y(test_attb, test_attp, test_control)

print(train_x[0])
print(train_y.shape)
print(train_x.shape)
print(train_x.shape)
print(test_x.shape)

set_1_train.tsv
set_1_dev.tsv
set_1_test.tsv
17406
18833
36239
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 0 1]
 [0 1 0 0]
 [

The code for this model was adapted from the following source: https://www.tensorflow.org/tutorials/keras/classification

In [0]:
def get_model(l1_val=None, middle_layer_val=128, middle_layer_activation="relu"):
    model_to_return = keras.Sequential([
        keras.layers.Flatten(input_shape=(160, 4))
    ])
    if l1_val is not None:
        model_to_return.add(keras.layers.Dense(middle_layer_val, input_dim=160,
                                               activation=middle_layer_activation,
                                               activity_regularizer=regularizers.l1(l1_val)))
    else:
        model_to_return.add(keras.layers.Dense(middle_layer_val, activation=middle_layer_activation))

    model_to_return.add(keras.layers.Dense(3, activation='softmax'))
    return model_to_return

def fit_model(model, train_x_arr, train_y_arr, verbose=2, epochs=25):
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(train_x_arr, train_y_arr, epochs=epochs, shuffle=True, verbose=verbose)

def eval_model(model, test_x_arr, test_y_arr):
    return model.evaluate(test_x_arr,  test_y_arr, verbose=0)

In [0]:
model = get_model()
print(train_y.shape)
print(train_x[0])
print(train_x.shape)
print(train_x.shape)
print(test_x.shape)
fit_model(model, train_x, train_y, epochs=10)
dev_loss, dev_acc = eval_model(model, dev_x, dev_y)
print('Development  set accuracy:', dev_acc)
test_loss, test_acc = eval_model(model, test_x, test_y)
print('Test  set accuracy:', test_acc)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
(76573, 1)
[[0 1 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 0 1]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 

KeyboardInterrupt: ignored

In [0]:
predictions = model.predict(test_x)

The code for plotting the image of our one-hot encoded sequence was adapted from the following source: https://www.tensorflow.org/tutorials/keras/classification

In [0]:
def plot_image(i, predictions_array, true_label, img):
    """
    Plots a sample image from our dataset.
    """
    predictions_array, true_label, img = predictions_array, true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'
    print(true_label)
    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label[0]]),
                                color=color)

i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_y, test_x)
plt.show()
print(predictions.shape)

In [0]:
plt.figure()
plt.imshow(train_x[0])
plt.grid(False)
plt.show()

The code for plotting the image of our one-hot encoded sequence was adapted from the following source: https://www.tensorflow.org/tutorials/keras/classification

In [0]:
plt.figure(figsize=(45,45))
range_to_show = 20
for i in range(range_to_show):
    plt.subplot(4, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    item_to_get = len(train_x) - i if int(i / 5) % 2 == 1 else i
    plt.imshow(train_x[item_to_get])
    plt.xlabel(class_names[train_y[item_to_get][0]])
plt.show()

## Regularization and Data Augmentation

We now have a training accuracy of 99.3% and a test accuracy of 85.6%. We see that we have high variance and perhaps are overfitting our training set.

In [0]:
def get_hyperparameters(simple=False):
    """
    :param simple: Whether or not to return only two hyperparameter configurations. For debugging.
    :return: Several hyperparamter configurations consisting of varying combinations of activation functions, hidden unit amounts, and l1 regularization constants.
    """
    activations = ["relu", "sigmoid"]
    layer_vals = [8, 32, 64, 128]
    l1_vals = np.random.exponential(scale=0.015, size=(5,)).reshape(5, 1)
    hyperparameters_to_return = []
    print(l1_vals)
    if simple:
        return [{
                    'activation': activations[0], 
                    'layer_val': 128,
                    'l1': 0.05
                },
                {
                    'activation': activations[1], 
                    'layer_val': 128,
                    'l1': 0.05
                }]

    for act in activations:
        for layer_val in layer_vals:
            for l_val in l1_vals:
                hyperparameters_to_return.append({
                    'activation': act, 
                    'layer_val': layer_val,
                    'l1': l_val
                })
    return hyperparameters_to_return

In [0]:
dev_accuracies = []
hyperparameters = get_hyperparameters(simple=False)
print(hyperparameters)
print("We are testing " + str(len(hyperparameters)) + ".")
for param in hyperparameters:
    print(param)
for hyper_params in hyperparameters:
    model_extended = get_model(l1_val=hyper_params['l1'],
                               middle_layer_val=hyper_params['layer_val'], 
                               middle_layer_activation=hyper_params['activation'])
    fit_model(model_extended, 
              train_x, 
              train_y,
              verbose=0)
    dev_loss, dev_acc = eval_model(model_extended, dev_x, dev_y)
    dev_accuracies.append(dev_acc)
    print(dev_accuracies)
    print(hyper_params)
    print('Development set accuracy:', dev_acc)

dev_accuracies = np.array(dev_accuracies)
print(dev_accuracies)
print(max(dev_accuracies))
index_of_hyper_to_get = dev_accuracies.argmax(axis=0)
print(hyperparameters[index_of_hyper_to_get])

None of these configurations resulted in an improved development set accuracy.

## CNN

The code for the CNN was adapted from the following source: https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/

In [0]:
def create_model(input_shape, kernel_size=3):
    print("kernel size = " + str(kernel_size))
    model_to_return = Sequential()
    convo_add = Conv2D(64, kernel_size=kernel_size, padding='same', activation='relu', input_shape=(input_shape[1], input_shape[2], 1))
    model_to_return.add(convo_add)
    model_to_return.add(Conv2D(32, kernel_size=kernel_size, padding='same',activation='relu'))
    model_to_return.add(MaxPooling2D(pool_size=(2, 2)))
    
    model_to_return.add(Conv2D(64, kernel_size=kernel_size, padding='same', activation='relu'))
    model_to_return.add(Conv2D(64, kernel_size=kernel_size, padding='same', activation='relu'))
    model_to_return.add(MaxPooling2D(pool_size=(2, 2)))
    model_to_return.add(Dropout(0.25))
    
    model_to_return.add(Flatten())
    model_to_return.add(Dense(512, activation='relu'))
    model_to_return.add(Dropout(0.5))
    model_to_return.add(Dense(3, activation='softmax'))
    return model_to_return

In [0]:
train_x_cnn, train_y_cnn = np.array(train_x), np.array(train_y)
print(train_x[0])
print(train_x_cnn.shape)
print(train_y_cnn.shape)
train_x_cnn = train_x_cnn.reshape(train_x.shape[0], train_x.shape[1], train_x.shape[2], 1)
model = create_model(train_x_cnn.shape)

[[0 0 1 0]
 [0 0 1 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 0 1]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]

In [0]:
fit_model(model, train_x_cnn, train_y_cnn, epochs=10)

Epoch 1/10
 - 22s - loss: 0.3802 - acc: 0.8550
Epoch 2/10
 - 20s - loss: 0.3697 - acc: 0.8574
Epoch 3/10
 - 20s - loss: 0.3554 - acc: 0.8653
Epoch 4/10
 - 20s - loss: 0.3486 - acc: 0.8667
Epoch 5/10
 - 20s - loss: 0.3401 - acc: 0.8711
Epoch 6/10
 - 20s - loss: 0.3288 - acc: 0.8738
Epoch 7/10
 - 20s - loss: 0.3255 - acc: 0.8770
Epoch 8/10
 - 20s - loss: 0.3138 - acc: 0.8808
Epoch 9/10
 - 20s - loss: 0.3091 - acc: 0.8832
Epoch 10/10
 - 20s - loss: 0.3078 - acc: 0.8831


In [0]:
dev_x_cnn = dev_x.reshape(dev_x.shape[0], dev_x.shape[1], dev_x.shape[2], 1)
dev_loss, dev_acc = eval_model(model, dev_x_cnn, dev_y)
print('Development set accuracy:', dev_acc)

test_x_cnn = test_x.reshape(test_x.shape[0], test_x.shape[1], test_x.shape[2], 1)
test_loss, test_acc = eval_model(model, test_x_cnn, test_y)
print('Test set accuracy:', test_acc)

Development set accuracy: 0.817085020175347
Test set accuracy: 0.8234214696805405


#### The visualization for the classes was adapted from the following source: https://jacobgil.github.io/deeplearning/class-activation-maps

In [0]:
def visualize_class_activation_map():
    original_img = dev_x_cnn[0]
    width, height, _ = original_img.shape # original_img.shape
    
    img = np.array([np.transpose(np.float32(original_img), (2, 0, 1))])
    
    class_weights = model.layers[-1].get_weights()[0]
    
    get_output = K.function([model.layers[0].input], \
                [model.layers[len(model.layers) - 1].output, 
    model.layers[-1].output])
    [conv_outputs, predictions] = get_output([img])
    conv_outputs = conv_outputs[0, :, :, :]

    # Class activation map.
    cam = np.zeros(dtype = np.float32, shape = conv_outputs.shape[1:3])
    target_class = 1
    for i, w in enumerate(class_weights[:, target_class]):
            cam += w * conv_outputs[i, :, :]

model.summary()

layer_outputs = [layer.output for layer in model.layers[:12]]

from keras import models
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)

img_tensor = np.expand_dims(dev_x_cnn[10], axis=0)
img_tensor = img_tensor / 255
img_tensor_to_show = img_tensor[0,:,:,0:1].reshape(160,4)
print(img_tensor_to_show.shape)
plt.imshow(img_tensor_to_show)
plt.show()

activations = activation_model.predict(img_tensor) 
first_layer_activation = activations[0]
print(first_layer_activation.shape)
plt.matshow(first_layer_activation[0, :, :, 32], cmap='viridis')

DEBUG = False

if DEBUG:
    last_layer_act = activations[3]
    print(last_layer_act.shape)
    plt.matshow(last_layer_act[:, :], cmap='viridis')
    
    last_layer_act = activations[1]
    print(last_layer_act.shape)
    plt.matshow(last_layer_act[0, :, :, 1], cmap='viridis')

    last_layer_act = activations[len(activations) - 2]
    print(last_layer_act.shape)
    plt.matshow(last_layer_act[:], cmap='viridis')


    last_layer_act = activations[len(activations) - 1]
    print(last_layer_act.shape)
    plt.matshow(last_layer_act[:], cmap='viridis')


In [0]:
model = create_model(train_x_cnn.shape)
fit_model(model, train_x_cnn, train_y_cnn, epochs=100)

In [0]:
dev_x_cnn = dev_x.reshape(dev_x.shape[0], dev_x.shape[1], dev_x.shape[2], 1)
dev_loss, dev_acc = eval_model(model, dev_x_cnn, dev_y)
print('Development set accuracy:', dev_acc)

In [0]:
# This represents our second round of hyperparameter testing

def hyperparameter_testing():
  print("set 1")
  print("center padding")
  cnn_epochs = []
  nn_epochs = [5]
  tests = [
           ["cnn", 3],
           ["cnn", 4],
           ["nn", None],
          ]
  print(train_x[0])
  results = []
  for test in tests:
    if test[0] == "nn":
      result = []
      for epoch in nn_epochs:
        model = get_model()
        fit_model(model, train_x, train_y, epochs=epoch)
        train_loss, train_acc = eval_model(model, train_x, train_y)
        dev_loss, dev_acc = eval_model(model, dev_x, dev_y)
        print('Development  set accuracy:', dev_acc)
        test_loss, test_acc = eval_model(model, test_x, test_y)
        print('Test  set accuracy:', test_acc)
        result.append([train_acc, dev_acc, test_acc])
      results.append(result)
    else:
      result = []
      

      for epoch in cnn_epochs:
        
        train_x_cnn, train_y_cnn = np.array(train_x), np.array(train_y)

        kernel_size = test[1]
        model = create_model(train_x_cnn.shape, kernel_size=kernel_size)

        print(train_x[0])
        print(train_x_cnn.shape)
        print(train_y_cnn.shape)
        train_x_cnn = train_x_cnn.reshape(train_x.shape[0], train_x.shape[1], train_x.shape[2], 1)

        fit_model(model, train_x_cnn, train_y_cnn, epochs=epoch)

        train_loss, train_acc = eval_model(model, train_x_cnn, train_y_cnn)

        dev_x_cnn = dev_x.reshape(dev_x.shape[0], dev_x.shape[1], dev_x.shape[2], 1)

        dev_loss, dev_acc = eval_model(model, dev_x_cnn, dev_y)
        print('Development set accuracy:', dev_acc)

        test_x_cnn = test_x.reshape(test_x.shape[0], test_x.shape[1], test_x.shape[2], 1)

        test_loss, test_acc = eval_model(model, test_x_cnn, test_y)
        print('Test set accuracy:', test_acc)
        result.append([train_acc, dev_acc, test_acc])
      results.append(result)
  return results

results = hyperparameter_testing()
print(results)

set 1
center padding
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 0 1]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 0]


In [0]:
train_x_cnn, train_y_cnn = np.array(train_x), np.array(train_y)

best_model = create_model(train_x_cnn.shape, kernel_size=3)

print(train_x[0])
print(train_x_cnn.shape)
print(train_y_cnn.shape)
train_x_cnn = train_x_cnn.reshape(train_x.shape[0], train_x.shape[1], train_x.shape[2], 1)

fit_model(best_model, train_x_cnn, train_y_cnn, epochs=25)

train_loss, train_acc = eval_model(best_model, train_x_cnn, train_y_cnn)

dev_x_cnn = dev_x.reshape(dev_x.shape[0], dev_x.shape[1], dev_x.shape[2], 1)

dev_loss, dev_acc = eval_model(best_model, dev_x_cnn, dev_y)
print('Development set accuracy:', dev_acc)

test_x_cnn = test_x.reshape(test_x.shape[0], test_x.shape[1], test_x.shape[2], 1)

test_loss, test_acc = eval_model(best_model, test_x_cnn, test_y)
print('Test set accuracy:', test_acc)

[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 1 0]

In [0]:
best_model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_61 (Conv2D)           (None, 160, 4, 64)        640       
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 160, 4, 32)        18464     
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 80, 2, 32)         0         
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 80, 2, 64)         18496     
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 80, 2, 64)         36928     
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 40, 1, 64)         0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 40, 1, 64)       

In [0]:
best_model.save("triple.h5")
from google.colab import files
files.download('triple.h5')